<a href="https://colab.research.google.com/github/Yohk2291/MachineLearning/blob/master/image_classification/1_3_fine_tuning_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
!git clone https://github.com/Yohk2291/MachineLearning.git
%cd MachineLearning/
%cd image_classification/

Cloning into 'MachineLearning'...
remote: Enumerating objects: 65, done.
remote: Counting objects: 100% (65/65), done.
remote: Compressing objects: 100% (60/60), done.
remote: Total 65 (delta 22), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (65/65), done.
/content/MachineLearning/image_classification/MachineLearning
/content/MachineLearning/image_classification/MachineLearning/image_classification


In [0]:
import os
import urllib.request
import zipfile

In [0]:
# フォルダ「data」が存在しない場合は作成する
data_dir = "./data/"
if not os.path.exists(data_dir):
    os.mkdir(data_dir)

In [0]:
# アリとハチの画像データをダウンロードし解凍
# PyTorchのチュートリアルで用意されているものです
# https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html

url = "https://download.pytorch.org/tutorial/hymenoptera_data.zip"
save_path = os.path.join(data_dir, "hymenoptera_data.zip")

if not os.path.exists(save_path):
    urllib.request.urlretrieve(url, save_path)

    # ZIPファイルを読み込み
    zip = zipfile.ZipFile(save_path)
    zip.extractall(data_dir)  # ZIPを解凍
    zip.close()  # ZIPファイルをクローズ

    # ZIPファイルを消去
    os.remove(save_path)

## 各種パッケージのインポート

In [0]:
import glob
import os.path as osp
import random
import numpy as np
import json 
from PIL import Image
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision
from torchvision import models, transforms


In [0]:
# 乱数のシードを設定
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)

In [24]:
print("PyTorch Version", torch.__version__)
print("Torchvision Version", torchvision.__version__)

PyTorch Version 1.1.0
Torchvision Version 0.3.0


## Datasetの作成

In [0]:
# 訓練時と推論時で異なる前処理を実行
class ImageTransform():
  def __init__(self, resize, mean, std):
    self.data_transform = {
        'train' : transforms.Compose([
                transforms.RandomResizedCrop(resize, scale=(0.5, 1.0)), # 0.5-1.0の間で画像を拡大・縮小
                transforms.RandomHorizontalFlip(), # 画像の左右を50%の確率で反転
                transforms.ToTensor(), # テンソルに変換
                transforms.Normalize(mean, std) # 標準化
                ]),
        'val' : transforms.Compose([
               transforms.Resize(resize), # 短辺の長さがresizeの大きさになる
               transforms.CenterCrop(resize), # 画像の中心をresize * resizeで切り取る
               transforms.ToTensor(), # テンソルに変換
               transforms.Normalize(mean, std) # 標準化
               ])
    }
    
  def __call__(self, img, phase='train'):
    return self.data_transform[phase](img) # phaseでtrain or val を指定

In [0]:
# アリとハチの画像へのファイルパスのリストを作成する


def make_datapath_list(phase="train"):
    """
    データのパスを格納したリストを作成する。

    Parameters
    ----------
    phase : 'train' or 'val'
        訓練データか検証データかを指定する

    Returns
    -------
    path_list : list
        データへのパスを格納したリスト
    """

    rootpath = "./data/hymenoptera_data/"
    target_path = osp.join(rootpath+phase+'/**/*.jpg')
    print(target_path)

    path_list = []  # ここに格納する

    # globを利用してサブディレクトリまでファイルパスを取得する
    for path in glob.glob(target_path):
        path_list.append(path)

    return path_list


In [27]:
# 実行
train_list = make_datapath_list(phase="train")
val_list = make_datapath_list(phase="val")

./data/hymenoptera_data/train/**/*.jpg
./data/hymenoptera_data/val/**/*.jpg


In [0]:
class HymenopteraDataset(data.Dataset):
  """
    アリとハチの画像のDatasetクラス。PyTorchのDatasetクラスを継承。

    Attributes
    ----------
    file_list : リスト
        画像のパスを格納したリスト
    transform : object
        前処理クラスのインスタンス
    phase : 'train' or 'test'
        学習か訓練かを設定する。
    """

  def __init__(self, file_list, transform=None, phase='train'):
    self.file_list = file_list  # ファイルパスのリスト
    self.transform = transform  # 前処理クラスのインスタンス
    self.phase = phase  # train or valの指定

  def __len__(self):
    '''画像の枚数を返す'''
    return len(self.file_list)

  def __getitem__(self, index):
    '''前処理をした画像のTensor形式のデータとラベルを取得'''
    # index番目の画像をロード
    img_path = self.file_list[index]
    img = Image.open(img_path)  # [高さ][幅][色RGB]

    # 画像の前処理を実施
    img_transformed = self.transform(img, self.phase)  # torch.Size([3, 224, 224])

    # 画像のラベルをファイル名から抜き出す
    if self.phase == "train":
      label = img_path[30:34]
    elif self.phase == "val":
      label = img_path[28:32]

    # ラベルを数値に変更する
    if label == "ants":
      label = 0
    elif label == "bees":
      label = 1

    return img_transformed, label


In [0]:
# Dataset作成
size = 224
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)

train_dataset = HymenopteraDataset(
    file_list=train_list, transform=ImageTransform(size, mean, std), phase='train')

val_dataset = HymenopteraDataset(
    file_list=val_list, transform=ImageTransform(size, mean, std), phase='val')


## DataLoaderを作成

In [30]:
# ミニバッチのサイズ
batch_size = 32

# DataLoaderの作成
train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True)

val_dataloader = torch.utils.data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False)

# 辞書型変数にまとめる
dataloaders_dict = {"train": train_dataloader, "val": val_dataloader}


# 動作確認
batch_iterator = iter(dataloaders_dict["train"])
inputs, labels = next(
    batch_iterator)

print(inputs.size())
print(labels)

torch.Size([32, 3, 224, 224])
tensor([0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0,
        1, 0, 1, 1, 1, 1, 1, 0])


## ネットワークモデルの作成

In [31]:
# 学習済みVGG-16をロード
use_pretrained = True #学習済みパラメータを使用
net = models.vgg16(pretrained=use_pretrained)

# VGG-16の最後の出力層の出力ユニットをアリとハチの２つに付け替える 
net.classifier[6] = nn.Linear(in_features=4096, out_features=2) # out_features: 1000 -> 2

# 訓練モードに設定
net.train()
print(net)
print('ネットワーク設定完了：学習済みの重みをロードし、訓練モードに設定しました')

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (17): Conv2d

## 損失関数を定義

In [0]:
# クロスエントロピー誤差
criterion = nn.CrossEntropyLoss()

※　ここまで1-2_transfer_learningと共通

## 最適化手法の設定

In [35]:
# ファインチューニングで学習させるパラメータを格納
params_to_update_1 = []
params_to_update_2 = []
params_to_update_3 = []

# 学習させるパラメータ
update_param_names_1 = ["features"]
update_param_names_2 = ["classifier.0.weight", "classifier.0.bias", "classifier.3.weight", "classifier.3.bias"]
update_param_names_3 = ["classifier.6.weight", "classifier.6.bias"]

# 学習させるパラメータ以外は変化させない
for name, param in net.named_parameters():
  if update_param_names_1[0] in name:
    param.requires_grad = True
    params_to_update_1.append(param)
    print("params_to_update_1に格納:", name)

  elif name in update_param_names_2:
    param.requires_grad = True
    params_to_update_2.append(param)
    print("params_to_update_2に格納:", name)

  elif name in update_param_names_3:
    param.requires_grad = True
    params_to_update_3.append(param)
    print("params_to_update_3に格納:", name)

  else:
    param.requires_grad = False
    print("勾配計算なし:", name)


params_to_update_1に格納: features.0.weight
params_to_update_1に格納: features.0.bias
params_to_update_1に格納: features.2.weight
params_to_update_1に格納: features.2.bias
params_to_update_1に格納: features.5.weight
params_to_update_1に格納: features.5.bias
params_to_update_1に格納: features.7.weight
params_to_update_1に格納: features.7.bias
params_to_update_1に格納: features.10.weight
params_to_update_1に格納: features.10.bias
params_to_update_1に格納: features.12.weight
params_to_update_1に格納: features.12.bias
params_to_update_1に格納: features.14.weight
params_to_update_1に格納: features.14.bias
params_to_update_1に格納: features.17.weight
params_to_update_1に格納: features.17.bias
params_to_update_1に格納: features.19.weight
params_to_update_1に格納: features.19.bias
params_to_update_1に格納: features.21.weight
params_to_update_1に格納: features.21.bias
params_to_update_1に格納: features.24.weight
params_to_update_1に格納: features.24.bias
params_to_update_1に格納: features.26.weight
params_to_update_1に格納: features.26.bias
params_to_update_1に格納: f

In [0]:
# 最適化手法の設定(MomentumSGD)
optimizer = optim.SGD([{'params':params_to_update_1, 'lr': 1e-4},
                        {'params':params_to_update_2, 'lr': 5e-4},
                        {'params':params_to_update_3, 'lr': 1e-4}
                        ], momentum=0.9)

## 学習・検証

In [0]:
def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):

  # GPUが使えるかの確認
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  print("使用デバイス：", device)
  
  # ネットワークをGPUへ
  net.to(device)
  
  torch.backends.cudnn.benchmark = True


  # epochのループ
  for epoch in range(num_epochs):
    print('Epoch{}/{}'.format(epoch+1, num_epochs))
    print('-'*80)

    # epoch毎の学習と検証のループ
    for phase in ['train', 'val']:
      if phase == 'train':
        net.train() # 訓練モードにする
      else:
        net.eval() # 検証モードにする

      epoch_loss = 0.0 # epochの損失和
      epoch_corrects = 0 # epochの正解数

      # 学習前の性能を確かめるため、epoch=0での訓練は省略
      if (epoch == 0) and (phase == 'train'):
        continue
      
      # dataloaderからミニバッチを取り出す
      for inputs, labels in tqdm(dataloaders_dict[phase]):

        inputs = inputs.to(device)
        labels = labels.to(device)

        # optimizer を初期化
        optimizer.zero_grad()

        # 順伝播
        with torch.set_grad_enabled(phase == 'train'):
          outputs = net(inputs)
          loss = criterion(outputs, labels)
          _, preds = torch.max(outputs, 1)

          # 訓練時は逆伝播も
          if phase == 'train':
            loss.backward()
            optimizer.step()

          # lossの合計を更新
          epoch_loss += loss.item() * inputs.size(0)
          # 正解数の合計を更新
          epoch_corrects += torch.sum(preds == labels.data)

      # epoch毎のlossと正解率を出力
      epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
      epoch_acc = epoch_corrects.double() / len(dataloaders_dict[phase].dataset)

      print('{} Loss: {:.4f}, Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

In [38]:
# 学習・検証を実行
num_epochs = 5
train_model(net, dataloaders_dict, criterion, optimizer, num_epochs)

使用デバイス： cuda:0


  0%|          | 0/5 [00:00<?, ?it/s]

Epoch1/5
--------------------------------------------------------------------------------


  0%|          | 0/8 [00:00<?, ?it/s]

val Loss: 0.8070, Acc: 0.3595
Epoch2/5
--------------------------------------------------------------------------------


...

val Loss: 0.2145, Acc: 0.9412
Epoch5/5
--------------------------------------------------------------------------------


  0%|          | 0/5 [00:00<?, ?it/s]

train Loss: 0.1941, Acc: 0.9424


100%|██████████| 5/5 [00:01<00:00,  2.87it/s]

val Loss: 0.1754, Acc: 0.9477


## 学習したネットワークの保存・ロード

In [0]:
# ネットワークパラメータの保存
save_path = './weights_fine_tuning.pth'
torch.save(net.state_dict(), save_path)

In [41]:
# ネットワークパラメータのロード
load_path = './weights_fine_tuning.pth'
load_weights = torch.load(load_path)
net.load_state_dict(load_weights)

# GPU上で保存された重みをCPU上でロードする場合
'''load_weights = torch.load(load_path, map_location={'cuda:0': 'cpu'})
net.load_state_dict(load_weights)
'''

"load_weights = torch.load(load_path, map_location={'cuda:0': 'cpu'})\nnet.load_state_dict(load_weights)\n"